In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import re 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.impute import SimpleImputer

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
import spacy

In [2]:
data = pd.read_csv("final_merged_data.csv")
data.head(1)

,professionals_id,professionals_headline,professionals_industry,answers_question_id,questions_id,questions_title,questions_body
0,0c673e046d824ec0ad0ebe012a0673e4,NaN,NaN,e214acfbe6644d65b889a3268828db9d,e214acfbe6644d65b889a3268828db9d,anesthesiologist,sophomore anesthesiologist steps order anesthe...


In [3]:
## Grab a corpus of text as a numpy array instead of series
text = data.questions_body.values.astype('U')
text

array(['sophomore anesthesiologist steps order anesthesiologist fun doctor experience anesthesiologist ',
       'sophomore jobs field nurse fun day years schooling medicine babies ',
       'sophomore jobs field nurse fun day years schooling medicine babies ',
       ..., 'collage singer collage singer ',
       'mcat semester study time time-management ',
       'auto careers mechanics '], dtype='<U3906')

In [4]:
token = RegexpTokenizer(r'[a-z]+')

In [5]:
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1), tokenizer = token.tokenize)
text_counts = cv.fit_transform(text)
text_counts = text_counts.toarray()
vocab = cv.get_feature_names()
text_counts = pd.DataFrame(text_counts, columns=vocab) ## yields pandas dataframe
text_counts.sample(3) 


,aa,aaa,aadsas,aand,aant,aauw,ab,aba,abc,abduction,...,zip,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zoos,zymeria
24896,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7277,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Hm, some weird words here and there that remains after preprocessing. 

In [6]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(text)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names()

## eliminate vocab beginning with numbers
#vocab = [w for w in vocab if re.search(r'[a-z]', w)]
tfDF = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab) ## also yields pandas dataframe
tfDF.sample(3)

,12,12629,12education,12th,14,16,1872,22,23,24,...,zombie,zone,zoo,zookeeper,zookeepers,zoologist,zoology,zoos,zymeria,áreas
15367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Hm, more weird stuff that remains after preprocessing according to this method

In [7]:
temp = pd.merge(tfDF, data, left_index = True, right_index = True)

In [8]:
temp.shape
temp.sample(2)

,12,12629,12education,12th,14,16,1872,22,23,24,...,zoos,zymeria,áreas,professionals_id,professionals_headline,professionals_industry,answers_question_id,questions_id,questions_title,questions_body
37088,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,f00cc95e7cd149e989c278f9d74995f6,Life Transitions/Career Coach Vicki Tillman Co...,Professional Training,189883e6b9384ff68b05b4e64b566fa5,189883e6b9384ff68b05b4e64b566fa5,ways class,volunteer church school teacher aid material g...
5250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,151351bf5a454634acce04bacd803b31,"Results driven marketer, specializing in Sales...",Marketing and Advertising,616c7459627b48a1ab52e1f8334bee3a,616c7459627b48a1ab52e1f8334bee3a,job college,hastle living job search job business career j...


In [9]:
X = tfDF
y = data['professionals_headline']

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=2)
imp = SimpleImputer(missing_values=np.nan, strategy='median')
X = imp.fit_transform(X)

In [14]:
knn = KNeighborsClassifier(n_neighbors=31, weights='distance')
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)

ValueError: Input contains NaN

In [15]:
tfDF.isnull().values.any()

False